# ノック31

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

データを読み込む

In [2]:
use_log = pd.read_csv("use_log.csv")
use_log.head()

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [3]:
customer = pd.read_csv("customer_join.csv")
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


欠損値の確認

In [4]:
use_log.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [5]:
customer.isnull().sum()

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

# ノック32

クラスタリングを行う

まずは、mean,median,max,min,membership_periodを分ける

In [6]:
customer_clustering = customer[["mean","median","max","min","membership_period"]]
customer_clustering.head()

,mean,median,max,min,membership_period
0,4.833333,5.0,8,2,47
1,5.083333,5.0,7,3,47
2,4.583333,5.0,6,3,47
3,4.833333,4.5,7,2,47
4,3.916667,4.0,6,1,47


4つのグループにクラスタリングを行う。  
membership_periodは数値の範囲が大きく異なるので、全体を標準化したものを使用する。

In [7]:
scaler = StandardScaler()
customer_clustering_scaler = scaler.fit_transform(customer_clustering)

pred_cluster =  KMeans(n_clusters=4, random_state = 0).fit(customer_clustering)
customer_clustering["cluster"] = pred_cluster.labels_
customer_clustering.head()

C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,mean,median,max,min,membership_period,cluster
0,4.833333,5.0,8,2,47,3
1,5.083333,5.0,7,3,47,3
2,4.583333,5.0,6,3,47,3
3,4.833333,4.5,7,2,47,3
4,3.916667,4.0,6,1,47,3


# ノック33(ここから自身の解答)

クラスター毎の平均値を算出する。

In [11]:
# まずは列名を日本語に変更

customer_clustering.rename(columns = {"mean" : "月内平均値", "median" : "月内中央値","max" : "月内最大値", "min" : "月内最小値", "membership_period" : "会員期間"},inplace = True)
customer_clustering.columns

C:\Users\Takeshi\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['月内平均値', '月内中央値', '月内最大値', '月内最小値', '会員期間', 'cluster'], dtype='object')

In [9]:
customer_clustering.head()

,月内平均値,月内中央値,月内最大値,月内最小値,会員期間,cluster
0,4.833333,5.0,8,2,47,3
1,5.083333,5.0,7,3,47,3
2,4.583333,5.0,6,3,47,3
3,4.833333,4.5,7,2,47,3
4,3.916667,4.0,6,1,47,3


In [10]:
# クラスター・列毎の個数の確認

customer_clustering.groupby("cluster").count()

,月内平均値,月内中央値,月内最大値,月内最小値,会員期間
cluster,,,,,
0,1137,1137,1137,1137,1137
1,1118,1118,1118,1118,1118
2,899,899,899,899,899
3,1038,1038,1038,1038,1038


In [14]:
# クラスター毎の平均の算出

customer_clustering.groupby("cluster").mean()

,月内平均値,月内中央値,月内最大値,月内最小値,会員期間
cluster,,,,,
0,3.727674,3.482850,6.315743,1.697449,7.272647
1,5.567745,5.566190,8.245975,2.948122,22.266547
2,7.877769,7.818131,9.931034,5.904338,6.070078
3,4.635116,4.623314,7.196532,2.133911,38.863198


この結果から、ざっくりと以下のような考察が得られる。 
("クラスター"は省略。Ex.クラスター1 → 1)  
  
・会員期間...0,1は半年ほどのグループ。2は2年弱ほどで、3は3年強。  
・0は最もコミットメントが低い。  
・2は月内平均値が最も高く、4つの中で最もコミットメントが高い。  
・3は会員期間が最も長いにも関わらずコミットメントが際立って高い訳では無い。ここから、「月4 - 5回くらい(週1回くらい)でコンスタントに来る人が長続きする」という仮説が導き出せる。  